## Исследование данных о российском кинопрокате



По лору, заказчик этого исследования — Министерство культуры Российской Федерации.

Нам нужно изучить рынок российского кинопроката и выявить текущие тренды. При этом уделить внимание фильмам, которые получили государственную поддержку. Попробуем ответить на вопрос, насколько такие фильмы интересны зрителю.

Мы поработаем с данными, опубликованными на портале открытых данных Министерства культуры. Набор данных содержит информацию о прокатных удостоверениях, сборах и государственной поддержке фильмов, а также информацию с сайта КиноПоиск.

### Шаг 1. Откройте файлы с данными и объедините их в один датафрейм. 

Объедините данные таким образом, чтобы все объекты из датасета `mkrf_movies` обязательно вошли в получившийся датафрейм. 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df_movies = pd.read_csv('/datasets/mkrf_movies.csv')
df_shows = pd.read_csv('/datasets/mkrf_shows.csv')

In [ ]:
df_movies.info()

In [ ]:
df_movies.head(10)

*df_movies* состоит из 15 столбцов:

1. title - Название фильма - тип данных object
2. puNumber - Номер прокатного удостовеления - тип данных object
3. show_start_date - Дата начала проката - тип данных object
4. type - Тип кинематогрофа - тип данных object
5. film_studio - Студия - тип данных object
6. production_country - Страна производитель - тип данных object
7. director - Режисер - тип данных object
8. producer - Продюсер - тип данных object
9. age_restriction - Возрастное ограничение - тип данных object
10. refundable_support - Возвратная господдержка - тип данных float
11. nonrefundable_support - Невозвратная господдержка - тип данных float
12. budget - Бюджет - тип данных float
13. financing_source - Источник финасирования - тип данных object
14. ratings - Рейтинг - тип данных object
15. genres - Жанр - тип данных object

Интересная законормерность:
- столбец *budget*, исходя из общей информации, скорее всего относится только к фильмам, получавшим гос. поддержку. То есть, в нём отражены только бюджеты гос. поддержки выделенные на фильм

In [ ]:
df_shows.info()

In [ ]:
df_shows.head(10)

Таблица со сведениями о показах состоит из двух столбцов:

1. puNumber - Номер прокатного удостоверения - тип данных int
2. box_office - Сумма сборов в прокате - тип данных float

Таким образом, чтобы объединить два датафрейма, нам необходимо:

- Привести столбец *puNumber* к одинаковому типу данных. Для этого используем метод *to_numeric* с атрибутом *errors = coerce*, чтобы в случае возникновения пропусков или иных косяков, заменить их на *NaN*. 
- Объединить два датафрейма методом *merge*, указав параметр *how = left*

In [ ]:
df_movies['puNumber'] = pd.to_numeric(df_movies['puNumber'], errors = 'coerce')
df = df_movies.merge(df_shows, on = 'puNumber', how = 'left')

In [ ]:
df.info()

In [ ]:
df.head(10)

Всего в нашей таблице 16 столбцов

- title — название кино;
- puNumber — номер прокатного удостоверения;
- show_start_date — дата премьеры;
- type — тип кинематогрофа;
- film_studio — студия производитель;
- production_country — страна производитель;
- director — режиссёр;
- producer — продюсер;
- age_restriction — возрастное ограничение;
- refundable_support — объём возвратных средств гос. поддержки;
- nonrefundable_support — объём невозвратных средств гос. поддержки;
- financing_source — источник государственного финансирования;
- budget — веделенный гос. бюджет;
- ratings — рейтинг кино;
- genres — жанр;
- box_office — кассовые сборы.

### Шаг 2. Предобработка данных

Для начала приведём название столбца *puNumber* к корректному змеиному регистру

In [ ]:
df = df.rename(columns = {'puNumber': 'distribution_certificate_number'})

In [ ]:
df.columns

#### Шаг 2.1. Проверьте типы данных

- Проверьте типы данных в датафрейме и преобразуйте их там, где это необходимо.

In [ ]:
df.info()

Преобразуем дату начала показа к типу данных *datetime*

In [ ]:
df['show_start_date'] = pd.to_datetime(df['show_start_date'], format = '%Y-%m-%dT%H:%M:%S')

In [ ]:
df['ratings'].unique()

Создадим функцию, которая посимвольно разобьёт ячейку и положит все значения в список, если в списке будет присутствовать знак %, то заменим эту ячейку нулевым значением. Применим данную функцию к столбцу ratings

In [ ]:
def ratio(cell):
    result = list(cell)
    if '%' in result:
        return 0
    else:
        return cell

заполним пропуски и применим функцию

In [ ]:
df['ratings'] = df['ratings'].fillna('%').astype('object')
df['ratings'] = df['ratings'].apply(ratio)

In [ ]:
df['ratings'].unique()

In [ ]:
df['ratings'] = df['ratings'].astype('float')

In [ ]:
df.info()

Задаим столбцу box_office тип данных int, так как при типе object длинное число будет неудобно отображаться. В нём есть пропуски, поэтому заменим их на 0, чтобы они не мешали преобразованию

In [ ]:
df['box_office'] = df['box_office'].fillna(0).astype('int')

**Вывод**

Преобразовали столбцы и поменяли типы данных для удобства дальнейших исследований

#### Шаг 2.2. Изучите пропуски в датафрейме

- Объясните, почему заполнили пропуски определённым образом или почему не стали этого делать.

Вычислим где и сколько пропусков

In [ ]:
df.isna().sum()

Выведем долю и подсветим

In [ ]:
pd.DataFrame(df.isna().mean()).style.background_gradient('coolwarm')

Сделаем чуть более читабельно

In [ ]:
pd.DataFrame(round(df.isna().mean()*100, 2)).style.background_gradient('coolwarm')

Итак, наши пропуски:

- количество пропусков в столбцах distribution_certificate_number, film_studio, production_country, и director незначительно. Их можно отбросить
- количество пропусков в столбце producer составляет более 7 %. Их лучше заменить на значение *unknown*, чтобы не терять данные.
- количество пропусков в столбцах refundable_support, nonrefundable_support, budget и financing_source составляет более 95%. Они относятся только к фильмам, которые получили государственную поддержку. Количество пропусков одинаковое - вероятее всего, что данные по этим четырём стоблбцам из одного источника. Думаю, что заполнять эти пропуски не стоит, дабы не искажать данные, которые могут понадобиться нам позже.
- количество пропусков в столбце genres составляет 13%. Пропуски в этом столбце можно заменить на unknown.

In [ ]:
to_drop = ['distribution_certificate_number', 'film_studio', 'production_country', 'director']

In [ ]:
df.dropna(subset = to_drop, inplace = True)

In [ ]:
to_unknown = ['genres', 'producer']

In [ ]:
def unkn_own(df, column, value='unknown'):
    df[column] = df[column].fillna(value)

In [ ]:
for column in to_unknown:
    unkn_own(df, column)

In [ ]:
df.isna().sum()

**Вывод**

Были удалены из таблицы незначительное количество пропусков. Для отдельных столбцов пропущенные значения заполнены на 0 и unknown.

#### Шаг 2.3. Изучите дубликаты в датафрейме
- Проверьте, есть ли в данных дубликаты. Опишите причины, которые могли повлиять на появление дублей.

Проверим наличие явных дубликатов. Сначала приведём все значения категориальных столбцов к единому регистру. Для этого создадим список из названий столбцов и пройдём по нему циклом.

Теперь посчитаем сумму дубликатов

In [ ]:
print("Кол-во дубликатов: {}".format(df.duplicated().sum()))

Дубликаты не найдены

#### Шаг 2.4. Изучите категориальные значения

- Посмотрите, какая общая проблема встречается почти во всех категориальных столбцах;
- Исправьте проблемные значения в поле `type`.

Переберём список уникальных значений в наших категориальных столбцах с помощью цикла

In [ ]:
for column in object_columns_list:
    print(column)
    print(df[column].sort_values().unique())

Видим, что во многих значениях разных столбцов присутствуют лишние пробелы. Уберём их и вновь пройдём циклом по списку наших категориальных столбцов и применим к ним метод .str.strip(). Этот метод удаляет все пробелы, которые встречаются в начале и в конце строки

In [ ]:
for column in object_columns_list:
    df[column] = df[column].str.strip()

In [ ]:
for column in object_columns_list:
    print(column)
    print(df[column].sort_values().unique())

**Вывод**

Подправили категориальные значения, убрав лишние пробелы

#### Шаг 2.5. Проверьте количественные значения

- Проверьте, обнаружились ли в таких столбцах подозрительные данные. Как с такими данными лучше поступить?

Зададим округление до двух знаков после запятой и взглянем на наши количественные столбцы

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
df[['budget', 'refundable_support', 'nonrefundable_support',
    'box_office']].describe()

**Столбец budget включает в себя полный объём государственной поддержки, а потому должен содержать в себе сумму столбцов refundable_support и nonrefundable_support, как минимум. Видим, что в нём есть нулевые значения, что не правильно. Найдём значения, где общая сумма бюджета меньше, чем суммарная государственная поддержка и исправим на данную сумму.**

Проверим, какие строки удовлетворяют нашему условию и посчитаем их количество

In [ ]:
display(df.query('budget < (refundable_support + nonrefundable_support)'))
print("Кол-во строк: {}".format(df.query(
   'budget < (refundable_support + nonrefundable_support)')['budget'].count()))

Заменим аномальные значения на суммарную государственную поддержку.

In [ ]:
df.loc[df['budget'] < (df['refundable_support'] + df['nonrefundable_support']),
       'budget'] = df['refundable_support'] + df['nonrefundable_support']

In [ ]:
df[['budget', 'refundable_support', 'nonrefundable_support',
    'box_office']].describe()

**Вывод**

Нашли и исправили аномальные количественные значения. Больше нету "безбюджетных" фильмов

#### Шаг 2.6. Добавьте новые столбцы





- Создайте столбец с информацией о годе проката. Выделите год из даты премьеры фильма.

Выделим год проката из столбца release_date и сохраним его в новый столбец release_year

In [ ]:
df['release_year'] = df['show_start_date'].dt.year

- Создайте два столбца: с именем и фамилией главного режиссёра и основным жанром фильма. В столбцы войдут первые значения из списка режиссёров и жанров соответственно.

Создадим функцию, которая будет брать первое значние (до запятой) из столбцов director и genres и применим её к нашим двум столбцам, чтобы получить новые два.

In [ ]:
def split(cell):
    result = cell.split(',')
    return result[0]

Получим имя режиссёра

In [ ]:
df['main_director'] = df['director'].apply(split)

Получим название жанра

In [ ]:
df['main_genre'] = df['genres'].apply(split)

- Посчитайте, какую долю от общего бюджета фильма составляет государственная поддержка.

In [ ]:
df['support'] = (df['refundable_support'] +
                       df['nonrefundable_support']) / df['budget']

In [ ]:
df.sample(10)

In [ ]:
df['support'].describe()

Среднее и медианное значение доли государственной поддержки составляет более половины: 54% и 61% соответственно. Минимум: 4%. Максимум: 100%
То есть, если гос. поддержка пристутсвует, то она чаще всего превышает половину бюджета кинопроизведения

**Вывод**

В данном разделе добавили новые столбцы, которые пригодятся нам в дальнейшем исследовании

**Вывод по шагу:**

Исправили некорректное название столбца. Поработали с пропусками, приведя датафрейм к приемлемым для исследования значениям. Далее заменили типы данных в столбцах для удобства будущих рассчётов. Затем нашли и исправили аномальные количественные значения и убрали лишние пробелы в категориальных значениях. И под конец добавили несколько новых столбцов, от которых в дальнейшем будем отталкиваться.

Датафрейм приведён в тот вид, который позволяет нам провести исследование и ответить на интересующие нас вопросы.

### Шаг 3. Проведите исследовательский анализ данных


#### Прокатные сборы по годам

- Посмотрите, сколько фильмов выходило в прокат каждый год. Обратите внимание, что данные о прокате в кинотеатрах известны не для всех фильмов. Посчитайте, какую долю составляют фильмы с указанной информацией о прокате в кинотеатрах.

In [ ]:
df.plot.hist(
             y = 'release_year', grid = True, title = 'Прокатные сборы по годам', bins = 10,
             range = (2010, df['release_year'].max()),
             figsize = (10, 6)
)
plt.xlabel('Год')
plt.ylabel('Количество')
plt.show()

Выведем количество релизов по годам

In [ ]:
print("Кол-во по годам:\n{}".format(df['release_year'].value_counts()))

Посмотрим долю фильмов, которые имеют информацию по сборам.

In [ ]:
print("Доля фильмов с информацией по сборам:\n{:.2%}"
      .format(df[df['box_office'] > 1]['box_office'].count() /
              df['box_office'].count()))

*Подитог:*

Лидер по количеству фильмов вышедших в прокат - 2010 год. 2019 на втором месте.
При этом информацию о сборах удалось обнаружить только у 41.98% кинопроизведений

#### Динамика сборов проката

- Изучите, как менялась динамика проката по годам. В каком году сумма сборов была минимальной? А максимальной?

Сначала посмотрим собственно суммы сборов по годам

In [ ]:
box_office_per_year = df.pivot_table(index = 'release_year', values = 'box_office', aggfunc = 'sum')
box_office_per_year

*Подитог:*

Видим, что до 2014(а скорее даже до 2015) у нас маловато данных. Однако по периоду с 2015 по 2019 можем наблюдать, что в среднем идёт постепенный рост выручки со сборов в кинопрокате. Значит ли это, что люди с каждым годом чаще всё ходили в кино? Да. Можем ли мы пронозировать дальнейший рост походов в кинотеатр. Вряд ли. Особенно учитывая рост популярности различных онлайн-площадок для просмотра кино(таких как Netflix, Оkko или Кинопоиск)

#### Средняя и медианная сумма сборов

- С помощью сводной таблицы посчитайте среднюю и медианную сумму сборов для каждого года.

Тут нам нужно быть осторожными, т.к. при расчётах нам могут помешать нулевые значения сборов. Поэтому сначала отфильтруем данные по фильмам, где сборы были больше нуля

In [ ]:
box_office_not_null = df.query('box_office > 0')

А теперь уже выведем нужную нам таблицу

In [ ]:
box_office_per_year_not_null = box_office_not_null.pivot_table(
    index = 'release_year',
    values = 'box_office',
    aggfunc = ['sum', 'mean', 'median']
    )
box_office_per_year_not_null

*Подитог:*

Наибольшее среднее видим в 2017 году. Тот же чемпион и по медианным показателям. 2018 - оба раза второй.

#### Влияние возратсного ограничения

- Определите, влияет ли возрастное ограничение аудитории («6+», «12+», «16+», «18+» и т. д.) на сборы фильма в прокате в период с 2015 по 2019 год? Фильмы с каким возрастным ограничением собрали больше всего денег в прокате? Меняется ли картина в зависимости от года? Если да, предположите, с чем это может быть связано.

Нам снова необходимо отфильтровать данные больше 0. Однако на этот раз, нас также инетерсуют только те кинополотна, которые выходили в прокат с 2015 по 2019. Снова отфильтруем первоначальный датафрейм

In [ ]:
box_office_age_restriction = df.query('(box_office > 0) & (release_year > 2014)')

А теперь уж строим сводную таблицу

In [ ]:
box_office_per_year_age_restriction = box_office_age_restriction.pivot_table(
    index = ['release_year', 'age_restriction'],
    values = 'box_office',
    aggfunc = ['sum']
    )
box_office_per_year_age_restriction

*Подитог:*

"0+" в пролёте по всем годам. Видимо совсем юная аудитория предпочитает другие способы проведения досуга, нежели поход в кино.

"18+" тоже не шибко блещет. Скорее всего в эту категорию в большинстве своём входят фильмы жанра "Ужасы", "Триллер" и подобное. А это весьма ограниченная аудитория, по сравнению со многими другими жанрами. 

"6+" часто оказывались 3ми, однако с 2017 года идёт явный рост в этом сегменте. А в 2019 так и вовсе выходит на 1ое место.

"12+" и "16+" как правило идут "ноздря в ноздрю" в лидерах кинопроката, хотя в выбранный период у "16+" больше почётных первенств.

Что примечательно, "6+", "12+" и "16+" практически выровнялись в 2019. Трудно предположить, с чем это может быть связано. Возможно подростки, являющиеся целевой аудиторией кино с рейтингом "12+" и "16+" стали чаще смотреть кино в интренете и качать его на торрентах. Что, в свою очередь, помогло кинопроизведениям с рейтингом "6+" догнать их, и даже выбиться в лидеры. А может в кино просто стали чаще "крутить мультики".

**Вывод**

Итак, что мы выяснили:

1. Среди почти половины фильмов, у которых есть данные по сборам, лидируют вышедшие в 2010 году. Не понятно почему, при общем росте прибылей из года в год, в лидерах 2010. Может отчётность в 2010 была прозрачней и доступней;

2. До 2015 статистика по суммам сборов видимо собиралась не очень хорошо. Но уже с 2015 видим плавное увеличение показателей общих сборов;

3. При вышеуказанном росте прибыли, наибольшее среднее и медианное значение по сборам приходится на 2017 год. Просто интересный факт: В 2016 Netflix запустился и стал доступен для регистрации пользователей в 190 странах мира. А с 2017 ещё и начал программу по поиску фрилансеров для перевода субтитров на более чем 20 языков(включая русский). Возможно тут есть закономерность. Но это не точно;

4. С возрастными ограничениями всё довольно закономерно. В топе стабильно идут "12+" и "16+", т.к. в этом сегменте наибольший разброс жанров(Что называется, на любой вкус и цвет). К тому же "0+" ещё не успел подсесть на иглу кинокультуры, а "18+" уже предпочитают смотреть кино дома не траятясь на дорогу до кинотеатра и попкорн. Да и разрешение на табак, алкоголь и женщин оказываются довольно губительны для интересов многих киноманов, я полагаю.

### Шаг 4. Исследуйте фильмы, которые получили государственную поддержку

На этом этапе нет конкретных инструкций и заданий — поищите интересные закономерности в данных. Посмотрите, сколько выделяют средств на поддержку кино. Проверьте, хорошо ли окупаются такие фильмы, какой у них рейтинг.

Тут начнём, как в некоторых пунктах до этого, с фильтрации датафрейма. Будем смотреть только те данные *budget* которых больше *0*

In [ ]:
df_with_gov_budget = df.loc[df.loc[:, 'budget'] > 0]

Взглянем на описательнцю статистику при помощи функции *describe*

In [ ]:
df_with_gov_budget['budget'].describe()

Видим, что минимальная гос. поддержка была равна 6 млн., а максимальная - более 2,3 млрд.!
Среднее значение составляет свыше 131 млн., а медианное порядка 71 млн.

**Посмотрим на рейтинги фильмов получавших гос. поддержку**

Отфильтруем от нулевых значений и построим гистограмму с количеством оценок

In [ ]:
df_with_budget_and_ratings = df_with_gov_budget.loc[
    df_with_gov_budget.loc[:, 'ratings'] > 0]

In [ ]:
df_with_budget_and_ratings.plot.hist(y='ratings', grid=True, title='Рейтинги', bins=10, range=(0, 10), figsize=(12, 5))
plt.xlabel('Рейтинг')
plt.ylabel('Количество оценок')
plt.show()

Посомтрим на распределние рейтингов у кино без и с гос. поддержкой

In [ ]:
fig, axes = plt.subplots(1,2, figsize = (14,6))
df['ratings'].plot(kind = 'hist',ax = axes[0], title = 'Распределение рейтингов кино')
axes[0].set_xlabel('Рейтинг')
df[df['budget'] > 0]['ratings'].plot(kind = 'hist',ax = axes[1], title = 'Распределение рейтингов кино с господдержкой')
axes[1].set_xlabel('Рейтинг')
plt.show()

Распределение в рейтингах выглядит крайне схоже

**Посмотрим на коммерческую успешность фильмов с государственной поддержкой**

In [ ]:
total_gos_box_office = df[df['budget'] > 0]['box_office'].sum()
total_gos_budget = df[df['budget'] > 0]['budget'].sum()
print(f'Всего собрали фильмы с гос. поддержкой: {total_gos_box_office:.1f}')
print(f'Всего бюджет фильмов с гос. поддержкой: {total_gos_budget:.1f}')

In [ ]:
total_gos_box_office = df[(df['budget'] > 0) & (df['nonrefundable_support'] != 0) & (df['refundable_support'] == 0)]['box_office'].sum()
total_gos_budget = df[(df['budget'] > 0) & (df['nonrefundable_support'] != 0) & (df['refundable_support'] == 0)]['budget'].sum()
print(f'Всего собрали фильмы только с невозвратной поддержкой: {total_gos_box_office:.1f}')
print(f'Всего бюджет фильмов только с невозвратной поддержкой: {total_gos_budget:.1f}')

In [ ]:
total_gos_box_office = df[(df['budget'] > 0) & (df['refundable_support'] != 0) & (df['nonrefundable_support'] == 0)]['box_office'].sum()
total_gos_budget = df[(df['budget'] > 0) & (df['refundable_support'] != 0) & (df['nonrefundable_support'] == 0)]['budget'].sum()
print(f'Всего собрали фильмы только с возвратной поддержкой: {total_gos_box_office:.1f}')
print(f'Всего бюджет фильмов только с возвратной поджеркой: {total_gos_budget:.1f}')

In [ ]:
success_films_total =  df[(df['budget'] > 0)
              & (df['box_office'] > df['budget'])]['title'].count()
total_gos_films = df[(df['budget'] > 0)]['title'].count()
success_part = success_films_total / total_gos_films
print(f'Из {total_gos_films} фильмов имели коммерческий успех {success_films_total}, что является {success_part:.1%} от общего числа.')

Как правило, фильмы с гос. поддержкой не имели коммерческого успеха и их сборы были меньше затраченных бюджетов. При этом фильмы, которые получали только возвратную поддержку заработали больше, чем потратили. Видимо, когда создателям и заказчикам сильно нужно, они могут заработать. А невозвратная поддержка, похоже, обрисовывает ситуацию с кино, целью которого могут являться просветительская деятельонсть или пропаганда тех или иных взглядов и ценностей. Ну или развитие киноиндустрии в целом(Хотя мне это кажется сомнительной статьёй расходов)

Ситуация выглядит не самой радужной. Является ли это следствием высокого уровня коммерческого кинематогрофа, по сравнению с отечественным кино получающим гос. поддержку. Или же к этому привели ошибки создателей нашего кино. В прочем, как говорил товарищ Сталин: "У каждой ошибки есть имя и фамилия". Давайте глянем на тех, чьи фильмы чаще других получали гос. поддержку

In [ ]:
df[df['budget'] > 0]['main_director'].value_counts().head(10)

Видим, что некоторые режисеры получали поддержку на свои фильмы несколько раз. Чемпион по убеждению чиновников в необходимости гос. финансирования именно его кино - Ренат Давльетьяров (5).

В целом можно сделать **вывод**, что господдержка в кино имеет смысл. Такое кино получают свою долю аудитории, через которую государство продвигает какие те свои идеи, несмотря на то, что зачастую такие фильмы не имеют коммерческого успеха. По крайней мере, с таким выводом согласился бы Р. Давлетьяров 

### Шаг 5. Напишите общий вывод

**Резюмируя всё вышеописанное:**

Сборы рынка кинопроката в период с 2015 по 2019 год демонстрировали стабильный рост. Видимо поход в кинотеатр для людей является одним из стандартных видов досуга, и они ходят туда независимо от того, какие фильмы идут в данный момент в прокате. Более того, возрастные ограничения не являются преградой для зрителя, т.к. в каждом сегменте представлено множетсво кинопроизведений на любой вкус и цвет. Хотя киноленты для детей и подростков (от 6 до 18 лет) собирают больше.

Что касается рейтингов фильмов снятых на государственные деньги, то они немного ниже рейтинга фильмов по всей выборке. Что важнее, коммерческого успеха картины с гос. поддержкой как правило не добиваются, если только это не кинополотна с возвратной поддержкой. Можно сделать вывод, что в том случае, когда поддержка рассматривается как инвестиция с целью получения прибыли - она окупается. В других случаях, видимо, поддержка имеет другую мотивацию. И есть мнение, что этой цели фильмы добиваются(В смысле кто то их всё же смотрит)

*Итого:* Фильмы с государственной поддержкой смотрят, хотя их рейтинг и немного ниже, чем рейтинг других фильмов. Полагаю, что практику выдачи государственных денег на съёмки можно продолжать. Хотя, я бы посоветовал подробней рассмотреть критерии отбора подобного кино, чтобы в дальнейшем оно было более успешным(как в плане сборов, так и в вопросе художественной ценности). Возможно стоит дать возможность новым малоизвестным кинотворцам(в прочем, это уже лирика и наше исследование этим впоросом не задавалось)